In [ ]:
import os
import unpopular
import numpy as np
import polars as pl
import lightkurve as lk
from astrocut import CutoutFactory
from scipy.signal import find_peaks
from astropy.coordinates import SkyCoord
from dask.distributed import wait, Client, LocalCluster

In [2]:
targets = pl.read_parquet("./data/targets.parquet").select("ID", "ra", "dec", "sector", "camera", "ccd")

In [4]:
def _count_harmonics(
    lc: lk.LightCurve, height: float = 0.15
) -> list[tuple[float, float]]:
    """Find the harmonics in the L-S periodogram of a given lightcurve.

    Args:
        lc (lk.LightCurve)
        height (float, optional): The minimum height of a peak as a fraction of the main harmonic. Defaults to 0.15.

    Returns:
        list[tuple[float, float]]: A list containing each harmonic as a tuple of period and power.
    """

    pg = lc.to_periodogram()
    period = pg.period_at_max_power

    if period.value >= 2:
        return []

    expected_harmonics = []
    for i in range(1, 9):
        expected_harmonics.append(period.value / i)

    peaks, properties = find_peaks(
        pg.power, distance=120, height=pg.max_power.value * height
    )

    peak_periods = [pg.period[idx].value for idx in peaks]

    found_harmonics = []
    for i, period in enumerate(peak_periods):
        in_range = 0.9 * expected_harmonics[i] <= period <= 1.1 * expected_harmonics[i]
        if in_range:
            found_harmonics.append((period, properties["peak_heights"][i]))
    # for harmonic in expected_harmonics:
    #     for idx, period in enumerate(peak_periods):
    #         if 0.9 * harmonic <= period <= 1.1 * harmonic:
    #             found_harmonics.append((period, properties["peak_heights"][idx]))
    #             break



    return found_harmonics

def is_complex(lc: lk.LightCurve) -> bool:
    """Check if a given lightcurve is complex by counting the number of harmonics."""
    return len(_count_harmonics(lc)) >= 3

In [5]:
def make_lightcurve(tic, coords, sector, camera, ccd):
    cube_cutter = CutoutFactory()

    cube_file = f"s3://stpubdata/tess/public/mast/tess-s{str(sector).zfill(4)}-{camera}-{ccd}-cube.fits"
    cutout = cube_cutter.cube_cut(cube_file, coordinates=coords, cutout_size=50, verbose=True, threads="auto")

    s = unpopular.Source(cutout, remove_bad=True)
    s.set_aperture(rowlims=[25, 26], collims=[25, 26])
    
    s.add_cpm_model(exclusion_size=5, n=64, predictor_method="similar_brightness")
    s.set_regs([0.1])
    s.holdout_fit_predict(k=100)

    apt_detrended_flux = s.get_aperture_lc(data_type="cpm_subtracted_flux")
    
    os.remove(cutout)
    return lk.TessLightCurve(time=s.time, flux=apt_detrended_flux)

In [6]:
def process_target(target):
    coords = SkyCoord(target["ra"], target["dec"], frame="icrs", unit="deg")
    lc = make_lightcurve(target["ID"], coords, target["sector"], target["camera"], target["ccd"])

    try:
        result = is_complex(lc)
    except:
        result = None
    
    return {**target, "result": result}

In [ ]:
num_workers = os.cpu_count()
cluster = LocalCluster(n_workers=num_workers, threads_per_worker=1, processes=True)
client = Client(cluster)

2025-03-12 08:18:01,919 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:55883' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'process_target-f37843a2081de9be5d6adcf56ea38342', 'process_target-61562b6ed050aad33ac809341d97949a', 'process_target-b62ec460e5be424ac2702c1dc32ac950', 'process_target-fe756d3e2b2575a11611c20545887968', 'process_target-3d8da8b198ed7614882cee24a9b527ce', 'process_target-a81ff1fd0cecdaf9c40a350ca7dcc7c7'} (stimulus_id='handle-worker-cleanup-1741781881.9189131')
2025-03-12 08:18:01,946 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:55865' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {'process_target-5938848867061a8970b5acf08aeede6e', 'process_target-754ad9991acee2ed3f40dd5667d0b491', 'process_target-07e211590abc33f3cdd8af2aa337fcd2', 'process_target-42d32a674505551670f5b55379740f06', 'process_target-986fdbad28b64c363fd0838a667f49f1', 'p

In [ ]:
futures = client.map(process_target, targets[0:1000].to_dicts())
wait(futures)